In [34]:
!pip install open_clip_torch[training] -q
!pip install --upgrade -q wandb

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)


In [35]:
import os
import sys
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt 
from urllib.request import urlretrieve

from kaggle_secrets import UserSecretsClient

import numpy as np
import pandas as pd
import torch
import wandb

In [36]:
class CFG:
    device = "cuda"
    seed = 42
    embedding_length = 384
    model_name = "coca_ViT-L-14"
    model_checkpoint_path = "/kaggle/input/open-clip-models/mscoco_finetuned_CoCa-ViT-L-14-laion2B-s13B-b90k.bin"

In [37]:
future_df = {"filepath":[], "title":[]}

In [38]:
# # Download the parquet table
# table_url = f'https://huggingface.co/datasets/poloclub/diffusiondb/resolve/main/metadata.parquet'
# urlretrieve(table_url, 'metadata.parquet')

# # Read the table using Pandas and truncate to the first 100k samples
# metadata_df = pd.read_parquet('metadata.parquet')[:150000]

# # Images path
# diffusiondf_path = ["/kaggle/input/diffusiondb-2m-part-0001-to-0100-of-2000",
#                     "/kaggle/input/diffusiondb-2m-part-0101-to-0200-of-2000",
#                     "/kaggle/input/diffusiondb-2m-part-0201-to-0300-of-2000",
#                     "/kaggle/input/diffusiondb-2m-part-0301-to-0400-of-2000",
#                     "/kaggle/input/diffusiondb-2m-part-0401-to-0500-of-2000",
#                     "/kaggle/input/diffusiondb-2m-part-0501-to-0600-of-2000",
#                     "/kaggle/input/diffusiondb-2m-part-0601-to-0700-of-2000",
#                     "/kaggle/input/diffusiondb-2m-part-0701-to-0800-of-2000",
#                     "/kaggle/input/diffusiondb-2m-part-0801-to-0900-of-2000",
#                     "/kaggle/input/diffusiondb-2m-part-0901-to-1000-of-2000",
#                     "/kaggle/input/diffusiondb-2m-part-1001-to-1100-of-2000",
#                     "/kaggle/input/diffusiondb-2m-part-1101-to-1200-of-2000",
#                     "/kaggle/input/diffusiondb-2m-part-1201-to-1300-of-2000",
#                     "/kaggle/input/diffusiondb-2m-part-1301-to-1400-of-2000",
#                     "/kaggle/input/diffusiondb-2m-part-1401-to-1500-of-2000",
#                     "/kaggle/input/diffusiondb-2m-part-1501-to-1600-of-2000",
#                     "/kaggle/input/diffusiondb-2m-part-1601-to-1700-of-2000",
#                     "/kaggle/input/diffusiondb-2m-part-1701-to-1800-of-2000",
#                     "/kaggle/input/diffusiondb-2m-part-1801-to-1900-of-2000",
#                     "/kaggle/input/diffusiondb-2m-part-1901-to-2000-of-2000"
#                    ]

In [39]:
# # Iterate throught each row of metadata dataframe 
# # and append its filepath and title to the future_df dictionary

# index = 0

# for _, row in metadata_df.iterrows():
#     group = index//100000
#     print(type(row['prompt']))
#     future_df["filepath"].append(f"{diffusiondf_path[group]}/{row['image_name']}")
#     future_df["title"].append(row['prompt'])
#     if index%100000 == 0:
#         print("Group",group)
#         print(f"{diffusiondf_path[group]}/{row['image_name']}")
#         print(row['prompt'])
#     index = index +1


In [40]:
import pandas as pd

df = pd.read_csv("/kaggle/input/gustavosta-stable-diffusion-prompts-sd2/train.csv")

future_df["filepath"] = df['image_path'].map(lambda x:x.lower())
future_df["title"] = df['Prompt']

In [41]:
# Path to save
train_data_path = "/kaggle/working/train_data.csv"

# Convert future_df dictionary to train_data.csv file with \t separator
pd.DataFrame.from_dict(future_df).to_csv(train_data_path, index=False, sep="\t")

In [42]:
wandb_api = UserSecretsClient().get_secret("wandb_api") 
wandb.login(key=wandb_api)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [43]:
!python -m training.main \
    --dataset-type "csv" \
    --train-data $train_data_path \
    --warmup 1000 \
    --batch-size 6 \
    --lr 1e-5 \
    --wd 0.1 \
    --epochs 1 \
    --workers 2 \
    --model $CFG.model_name \
    --pretrained $CFG.model_checkpoint_path \
    --report-to "wandb" \
    --coca-contrastive-loss-weight 0 \
    --coca-caption-loss-weight 1 \
    --log-every-n-steps 100

/bin/bash: /opt/conda/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2023-02-28,03:25:34 | INFO | Running with a single process. Device cuda:0.
2023-02-28,03:25:34 | INFO | Loaded coca_ViT-L-14 model config.
2023-02-28,03:25:41 | INFO | Loading pretrained coca_ViT-L-14 weights (/kaggle/input/open-clip-models/mscoco_finetuned_CoCa-ViT-L-14-laion2B-s13B-b90k.bin).
2023-02-28,03:25:45 | INFO | Model:
2023-02-28,03:25:45 | INFO | CoCa(
  (text): TextTransformer(
    (token_embedding): Embedding(49408, 768)
    (transformer): Transformer(
      (resblocks): ModuleList(
        (0): ResidualAttentionBlock(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ls_1): Identity()
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
         

Failed to sample metric: process no longer exists (pid=363)
Failed to sample metric: process no longer exists (pid=363)
Failed to sample metric: process no longer exists (pid=363)
Failed to sample metric: process no longer exists (pid=363)
Failed to sample metric: process no longer exists (pid=247)
Failed to sample metric: process no longer exists (pid=247)
Failed to sample metric: process no longer exists (pid=247)
Failed to sample metric: process no longer exists (pid=247)
Failed to sample metric: process no longer exists (pid=102)
Failed to sample metric: process no longer exists (pid=102)
Failed to sample metric: process no longer exists (pid=102)
Failed to sample metric: process no longer exists (pid=102)


In [44]:
# import os

# for path in df['image_path'].map(lambda x:x.lower()):
#     print(path)

Failed to sample metric: process no longer exists (pid=363)
Failed to sample metric: process no longer exists (pid=363)
Failed to sample metric: process no longer exists (pid=363)
Failed to sample metric: process no longer exists (pid=363)
Failed to sample metric: process no longer exists (pid=247)
Failed to sample metric: process no longer exists (pid=247)
Failed to sample metric: process no longer exists (pid=247)
Failed to sample metric: process no longer exists (pid=247)
Failed to sample metric: process no longer exists (pid=102)
Failed to sample metric: process no longer exists (pid=102)
Failed to sample metric: process no longer exists (pid=102)
Failed to sample metric: process no longer exists (pid=102)
Failed to sample metric: process no longer exists (pid=363)
Failed to sample metric: process no longer exists (pid=363)
Failed to sample metric: process no longer exists (pid=363)
Failed to sample metric: process no longer exists (pid=363)
Failed to sample metric: process no long